# CSV Processor Pro: Data Analysis Dashboard
This notebook provides an interactive interface to analyze and visualize your CSV data.

--- 
### ⚠️ 1. SETUP: THE "BULLETPROOF" FIX
Run this cell first! It will find your scripts and fix the connection automatically.

In [ ]:
import os
import sys
from pathlib import Path

# ---------------------------------------------------------------------------
# 1. AUTOMATIC ROOT DETECTION
# ---------------------------------------------------------------------------
def setup_project_paths():
    # Locate the folder containing 'scripts' and 'app'
    current = Path(os.getcwd()).resolve()
    root = None
    
    # Search upwards for the project root
    for _ in range(4):
        if (current / 'scripts').is_dir() and (current / 'app').is_dir():
            root = current
            break
        current = current.parent
        
    if root is None:
        print("❌ ERROR: Project root not found! Are you running this inside the 'csv_processor' folder?")
        return None
    
    root_str = str(root)
    if root_str not in sys.path:
        sys.path.insert(0, root_str)
    
    print(f"📂 Project Root Detected: {root_str}")
    return root

project_root = setup_project_paths()

# ---------------------------------------------------------------------------
# 2. IMPORT SCRIPTS
# ---------------------------------------------------------------------------
if project_root:
    try:
        # Import using the package structure
        from scripts.kpi_analyzer import calculate_kpis
        from scripts.visualizer import generate_visualizations
        print("✅ SUCCESS: 'scripts' package loaded correctly!")
    except ImportError as e:
        print(f"⚠️ Package import failed ({e}). Trying direct import...")
        # Fallback: Add 'scripts' folder directly to path
        scripts_dir = str((project_root / 'scripts').resolve())
        if scripts_dir not in sys.path:
            sys.path.insert(0, scripts_dir)
        try:
            import kpi_analyzer
            import visualizer
            calculate_kpis = kpi_analyzer.calculate_kpis
            generate_visualizations = visualizer.generate_visualizations
            print("✅ SUCCESS: Scripts loaded via direct path fallback!")
        except ImportError as e2:
            print(f"❌ FINAL ERROR: Could not load scripts. {e2}")

# ---------------------------------------------------------------------------
# 3. LOAD DATA
# ---------------------------------------------------------------------------
if project_root:
    data_file = project_root / 'data' / 'large_sample.csv'
    if data_file.exists():
        import pandas as pd
        import matplotlib.pyplot as plt
        import seaborn as sns
        df = pd.read_csv(str(data_file))
        print(f"✅ SUCCESS: Loaded {len(df)} rows of data.")
        display(df.head())
    else:
        print(f"❌ ERROR: Data not found at {data_file}. Please run 'scripts/generate_sample.py' first.")

### 📊 2. Key Performance Indicators (KPIs)

In [ ]:
if 'df' in locals():
    kpis = calculate_kpis(df)
    print("--- Summary KPIs ---")
    for k, v in kpis.items():
        print(f"{k.replace('_', ' ').title()}: {v:,.2f}" if isinstance(v, float) else f"{k.replace('_', ' ').title()}: {v}")

### 📈 3. Visualizations

In [ ]:
if 'df' in locals():
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(10, 6))
    sns.histplot(df['salary'], kde=True, color='green')
    plt.title('Overall Salary Distribution')
    plt.show()